# Week 2: Least Squares Fitting (cont.)

## Goals
- Plotting data in Python via [matplotlib](https://matplotlib.org/)
- Higher dimensional least squares fitting in Python

## Plotting with Matplotlib
The module [matplotlib](https://matplotlib.org/) is very powerful and customizable. 

We will plot the data points (ignoring the $i$ column) from the in-class exercise from Week 1.

In [ ]:
import pandas as pd 

df = pd.DataFrame({
    "x" : [1.0, 2.0, 3.0, 1.5, 3.5, 3.0, 4.0, 5.0],
    "y" : [1.0, 1.5, 2.0, 2.0, 3.0, 4.5, 2.0, 3.5]
})
print(df)

Pandas already incorporates matplotlib in its `plot` method for a data frame, but it's not what we want.

In [ ]:
df.plot()           # Using matplotlib in the backend

We will build a scatter plot from scratch.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.grid()                                   # Turns the grids on
ax.scatter(df["x"], df["y"], zorder=2)      # Builds a scatter plot 
plt.show()                                  # Displays the plot

### Least Squares Fitting

Now let's put a least squares line in the plot. 

First we need to compute the least squares line.

Recall 
$$
\begin{aligned}
    X &= \begin{pmatrix} 
        1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_n
    \end{pmatrix}, & 
    Y &= \begin{pmatrix} 
        y_1 \\ y_2 \\ \vdots \\ y_n
    \end{pmatrix} , & 
    B &= \begin{pmatrix}
        b_0 \\ b_1
    \end{pmatrix} ,
\end{aligned} 
$$
where $B = (X^{\mathrm{t}}X)^{-1} X^{\mathrm{t}}Y$, provided $X^{\mathrm{t}}X$ is invertible.

We will use the `numpy` matrix data structures.

In [ ]:
import numpy as np

X = np.array([[1]*len(df), df["x"]]).T      # The method T is the transpose
print(X)
Y = np.array([df["y"]]).T
print(Y)

Now we compute $B = (X^{\mathrm{t}}X)^{-1} X^{\mathrm{t}}Y$:

In [ ]:
# print(X.T @ X)
# print(np.linalg.inv(X.T @ X))                              # (X^t * X)^{-1}
# print(np.linalg.inv(X.T @ X) @ X.T @ X)

B = np.linalg.inv(X.T @ X) @ X.T @ Y
print(B)
b0 = B[0,0]                                     # The (0,0) entry of B
b1 = B[1,0]                                     # The (1,0) entry of B
print(f"b0 = {b0}\nb1 = {b1}")    		  		# Fancy printing

### Plotting the scatter plot and the line

Now we need to include the line $y=b_0+b_1x$ into our plot above. 

We just simply create both a scatter and a plot, and then we display the resulting plot.

Matplotlib simply connects points

In [ ]:
xs = np.linspace(0.9, 5.1, 2)
ys = b0 + b1*xs
print(f"xs = {xs}\nys = {ys}")

We can actually make the pandas plot work 😁

In [ ]:
fig, ax = plt.subplots()
df.plot(kind="scatter", x="x", y="y", ax=ax, zorder=3)
ax.plot(xs, ys, c="orange", zorder=2)
ax.grid()
ax.set_xticks([1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])
plt.show()

## Plane of best fit
Just as before, we can load our data from `./data/ex2.csv` using pandas.

In [ ]:
df = pd.read_csv("data/ex2.csv")
print(df)

Alternatively we can run the following code to load the data we need. To turn it "on" remove all of the `#` symbols.

In [ ]:
# pd.DataFrame({
#     "x_i1" : [278, 252, 344, 134, 215, 261, 131, 463, 167, 298, 230, 293, 290, 271, 385, 354],
#     "x_i2" : [36, 31, 35, 33, 35, 40, 39, 43, 46, 42, 60, 67, 37, 31, 63, 36],
#     "y_i" : [287, 256, 300, 182, 248, 271, 149, 411, 214, 291, 314, 352, 298, 252, 439, 328],
# })

To get the plane of best fit, we construct the relevant matrices and solve:
$$
\begin{aligned} 
    X &= \begin{pmatrix} 
        1 & x_{11} & x_{12} \\ 1 & x_{21} & x_{22} \\ \vdots & \vdots & \vdots \\ 1 & x_{n1} & x_{n2}
    \end{pmatrix}, & 
    Y &= \begin{pmatrix} 
        y_1 \\ y_2 \\ \vdots \\ y_n
    \end{pmatrix} , & 
    B &= \begin{pmatrix}
        b_0 \\ b_1 \\ b_2
    \end{pmatrix} .
\end{aligned}
$$

We will build $X$ by concatenating three columns:

In [ ]:
ones = np.array([[1] * 16]).T
xi1 = np.array([df["x_i1"]]).T
xi2 = np.array([df["x_i2"]]).T
# print(ones, xi1, xi2)
X = np.concatenate((ones, xi1, xi2), axis=1)
print(X)

In [ ]:
Y = np.array([df["y_i"]]).T
print(Y)

In [ ]:
B = np.linalg.inv(X.T @ X) @ X.T @ Y
print(B)

Therefore, our plane of best fit is approximately 
$$
    y = -11.3 + 0.7x_1 + 2.6x_2. 
$$

## Plotting a 3D scatter with a plane

First we plot the data points in $\mathbb{R}^3$. [Matplotlib example](https://matplotlib.org/stable/gallery/mplot3d/scatter3d.html) for reference.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(projection='3d')
ax.scatter(df["x_i1"], df["x_i2"], df["y_i"], c="blue")
ax.set_xlabel("x_1")
ax.set_ylabel("x_2")
ax.set_zlabel("y")
ax.set_box_aspect(aspect=None, zoom=0.9)

Now we put a plot of our plane of best fit onto the same axes. [Matplotlib example](https://matplotlib.org/stable/gallery/mplot3d/surface3d.html) for reference.

First we need to build up our plane.

In [ ]:
Xs = np.linspace(110, 490, 20)      # 20 points in interval [110, 490]
Ys = np.linspace(30, 68, 10)        # 10 points in interval [30, 68]
XXs, YYs = np.meshgrid(Xs, Ys)      # Makes a mesh -- essentially Xs x Ys.
# print(XXs, YYs)

Let's see what `Xs` and `Ys` look like.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(XXs, YYs, c="green")         # Mesh = green
ax.scatter(Xs, [30]*len(Xs), c="blue")  # Xs = blue
ax.scatter([110]*len(Ys), Ys, c="red")  # Ys = red

Now we will build our plane from this data.

In [ ]:
# Plot from before:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(projection='3d')
ZZs = B[0,0] + B[1,0]*XXs + B[2,0]*YYs      # y = b_0 + b_1*x_1 + b_2*x_2
ax.plot_surface(XXs, YYs, ZZs)

Now let's take the data and put them in the same plot! (Finally)

(This will be hard to see. We really need an animation...)

In [ ]:
# Plot from before:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(projection='3d')
ax.scatter(df["x_i1"], df["x_i2"], df["y_i"], c="orange")
ax.plot_surface(XXs, YYs, ZZs, alpha=0.5)
ax.set_xlabel("x_1")
ax.set_ylabel("x_2")
ax.set_zlabel("y")
ax.set_box_aspect(aspect=None, zoom=0.9)

Although the 3D plot is a bit hard to parse, there are ways around this---but we won't go there. 

There are ways to make animations, so one could rotate the camera angles judiciously in order to see how our plane fits the data. 